In [1]:
import numpy as np
import sys
import scipy.stats
import re

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../codes')

In [4]:
%pprint

Pretty printing has been turned OFF


In [5]:
from VMSfunctions.Common import *
from VMSfunctions.Chemicals import *
from VMSfunctions.Chromatograms import *
from VMSfunctions.MassSpec import *

# Mass Spec test

In [6]:
ps = load_obj('../models/peak_sampler_4_beers.p')
xcms_output = '../models/beer_ms1_peaks.csv.gz'
chromatograms = ChromatogramCreator(xcms_output)

c:\users\vinny\.virtualenvs\simulator-r22bo2kk\lib\site-packages\sklearn\base.py:253: UserWarning:

Trying to unpickle estimator KernelDensity from version 0.20.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.



0


../codes\VMSfunctions\Common.py:50: RuntimeWarning:

divide by zero encountered in double_scalars



5000
10000
15000
20000
25000
30000
35000
40000
45000


In [7]:
chemicals = ChemicalCreator(ps, chromatograms)

In [137]:
min_ms1_intensity = 2E5
min_rt = 3*60
max_rt = 21*60
dataset = chemicals.sample(min_rt, max_rt, min_ms1_intensity, 2, 3, "Unknown","Empirical", None, False, 0.01)

Warning ms_level > 3 not implemented properly yet. Uses ms_level = 2 information for now


In [ ]:
scan_parameters = {
    'isolation_windows': [[(0, 1e3)]], # TODO: change to dictionary?
    'ms_level': 1
}    
sampled_chemicals = chemicals.sample(min_rt, max_rt, min_ms1_intensity, 100, 2, "Unknown","Empirical",None, False)
mass_spec = IndependentMassSpectrometer(sampled_chemicals, scan_parameters)

In [ ]:
scan_windows = {
    2: [(0, 20), (0, 1e3)],
}
mass_spec._get_all_mz_peaks(dataset[0], 810, 2, [[(0, 1e3)]])

In [140]:
import pickle
hmdb = pickle.load(open('../../../Compounds/hmdb_compounds.p','rb'))

In [156]:
min_ms1_intensity = 2E5
min_rt = 3*60
max_rt = 21*60
dataset = chemicals.sample(min_rt, max_rt, min_ms1_intensity, 100, 3, "Known","Empirical", None, False, 0.01, hmdb)

Warning ms_level > 3 not implemented properly yet. Uses ms_level = 2 information for now
['C50H79O10P', 'C70H136O17P2', 'C69H134O17P2', 'C50H96O6', 'C60H116O6', 'C33H22O22', 'C44H84O6', 'C67H130O17P2', 'C4H8O', 'C59H96O6', 'C87H170O17P2', 'C52H100O6', 'C73H142O17P2', 'C75H146O17P2', 'C61H102O6', 'C54H104O6', 'C40H70NO10P', 'C69H134O17P2', 'C61H118O17P2', 'C69H134O17P2', 'C89H154O17P2', 'C72H136O17P2', 'C63H122O17P2', 'C8H2F17NO2S', 'C27H40O3', 'C54H104O6', 'C75H146O17P2', 'C49H94O6', 'C44H34O22', 'C25H24O12', 'C69H134O6', 'C19H30O2', 'C56H40O13S', 'C27H28O18', 'C67H130O17P2', 'C68H132O17P2', 'C78H148O17P2', 'C54H104O6', 'C4H4N2O2', 'C15H22O2', 'C37H72O5', 'C58H112O17P2', 'C55H98O6', 'C79H154O17P2', 'C65H104O6', 'C84H164O17P2', 'C67H130O17P2', 'C43H82O6', 'C13H12O4', 'C59H114O17P2', 'C51H94O6', 'C65H108O6', 'C23H26O11', 'C15H16O4', 'C36H66O5', 'C55H73N9O10S2', 'C63H122O17P2', 'C78H148O17P2', 'C82H160O17P2', 'C10H9NO5', 'C6H13NO2', 'C80H156O17P2', 'C51H90O6', 'C54H104O6', 'C19H18O2', 'C9

In [155]:
dataset[1]

KnownChemical - 'C53H104NO8P' rt=790.90 max_intensity=381743.92